Задание: Бейзлайн модель, которая работает на отложенной выборке (train_test_split) и validation.csv (различие в точности может быть большим, но главное попробовать любыми способами его уменьшить и зафиксировать эксперименты) - 3 балла

Воспользуюсь переводом валидации на английский.
Буду перебирать признаки  и печатать результаты тестирования на отложенной выборке, результате перевода валидации, полученном кодом:

from googletrans import Translator
translator = Translator()
validation['translation'] = validation['comment_text'].apply(lambda x: translator.translate(x).text)
validation.to_csv('validation_with_translation.csv')

Часть текстов не перевелась, поэтому отдельно буду считать на абсолютно всех текстах и только на переведенных

In [46]:
import pandas as pd
import sklearn
train1 = pd.read_csv('toxic-comment-train.csv')
validation = pd.read_csv('validation_with_translation.csv')

In [10]:
train1

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [5]:
validation_really_translated = validation.query('comment_text != translation')

In [53]:
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tf = TfidfVectorizer()

In [15]:
model = LogisticRegression()

In [16]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
tf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [24]:
texts = train1.comment_text
y = train1.toxic
texts_train, texts_test, y_train, y_test = sklearn.model_selection.train_test_split(texts, y)

texts_valid_all = validation.comment_text
y_valid_all = validation.toxic

texts_valid_translated = validation_really_translated.comment_text
y_valid_translated = validation_really_translated.toxic

In [28]:
len(texts_valid_translated)

5226

Так как я хочу не просто найти лучшие метрики, а посмотреть, как соотносятся результаты на трёх разных тестах, перебор буду писать вручную.

In [36]:
for ngram_range in [(1, 1), (1,3)]:
    for max_features in [750, 1500]:
        tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
        print(tfidf)
        X_train = tfidf.fit_transform(texts_train)
        X_test = tfidf.transform(texts_test)
        X_valid_all= tfidf.transform(texts_valid_all)
        X_valid_translated = tfidf.transform(texts_valid_translated)
        for C in [0.001, 0.5, 1, 5, 10, 100, 500, 1000]:
            print('C='+str(C))
            model = LogisticRegression(C=C)
            model.fit(X_train, y_train)
            english_prediction = model.predict(X_test)
            all_valid_prediction = model.predict(X_valid_all)
            translated_valid_prediction = model.predict(X_valid_translated)
            english_roc = sklearn.metrics.roc_auc_score(y_test, english_prediction)
            all_valid_roc = sklearn.metrics.roc_auc_score(y_valid_all, all_valid_prediction)
            translated_valid_roc = sklearn.metrics.roc_auc_score(y_valid_translated, translated_valid_prediction)
            print('english: ', english_roc, 'all_v: ', all_valid_roc, 'translated_v: ', translated_valid_roc)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=750,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.5 all_v:  0.5 translated_v:  0.5
C=0.5


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.7367581431022853 all_v:  0.5084999579685604 translated_v:  0.5069230134272507
C=1


...

english:  0.7626538043343252 all_v:  0.5087587515461566 translated_v:  0.5073010675764914
C=500


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.7625396274872878 all_v:  0.5087587515461566 translated_v:  0.5073010675764914
C=1000


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.7625396274872878 all_v:  0.5087587515461566 translated_v:  0.5073010675764914


При С близком к нулю результаты на английской выборке ужасные, при чуть-чуть больших значениях - уже 70+ и выше не становятся. На всей валидации результат чуть-чуть лучше, чем на выбранной только переведенной.
На переводе работает сильно хуже, чем на отложенной выборке. Попробую прогнать трейн через переводчик туда-обратно в надежде унифицировать используемую лексику

In [41]:
import googletrans
import random

In [48]:
from googletrans import Translator
translator = Translator()

In [49]:
def double_translation(english_text):
    if random.random() > 0.5:
        dest = 'de'
    else:
        dest = 'fr'
    translated = translator.translate(text=english_text, dest=dest).text
    new_english_text = translator.translate(translated).text
    return new_english_text

К сожалению, это не получилось запустить на всех данных, возьму сампл

In [47]:
sample = train1[:5000]

In [50]:
sample['bad_english'] = sample['comment_text'].apply(double_translation)

C:\Users\lizan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
sample.to_csv('train_with_double_translation.csv')

In [52]:
sample

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,bad_english
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation\nWhy were the changes made under m...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"D'aww! It matches this background color, which..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"Hey man, I'm not really trying to change the w..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"""\nMore\nI can't make any real suggestions for..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"You, sir, are my hero. Do you remember the pag..."
...,...,...,...,...,...,...,...,...,...
4995,0d39d2eaa40a6241,"""\n\nHello Marcruhwedell, and Welcome to Wikip...",0,0,0,0,0,0,"""\n\nHello Marcruhwedell and welcome to Wikipe..."
4996,0d3b57f3b2db3f03,"...that's why I did ....cheers, (talk · cont...",0,0,0,0,0,0,"... that's why I have ... cheers, (discussion ..."
4997,0d3bb1f12d90a6a2,"No, it's not a delayed reaction\n\nI just happ...",1,0,0,0,1,0,"No, this is not a delayed reaction\n\nI just r..."
4998,0d3be13abf1bec52,"""\n\nA slight difference with you\nI have to d...",0,0,0,0,0,0,"""\n\nA slight difference with you\nI have to d..."


Пересчитаю на оригинале на сампле, чтобы корректно сравнивать

In [54]:
train, test = sklearn.model_selection.train_test_split(sample)

original_train = train.comment_text
original_test = test.comment_text

bad_english_train = train.bad_english
bad_english_test = test.bad_english

y_train = train.toxic
y_test = test.toxic

texts_valid_all = validation.comment_text
y_valid_all = validation.toxic

texts_valid_translated = validation_really_translated.comment_text
y_valid_translated = validation_really_translated.toxic

In [55]:
for ngram_range in [(1, 1), (1,3)]:
    for max_features in [750, 1500]:
        tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
        print(tfidf)
        X_train = tfidf.fit_transform(original_train)
        X_test = tfidf.transform(original_test)
        X_valid_all= tfidf.transform(texts_valid_all)
        X_valid_translated = tfidf.transform(texts_valid_translated)
        for C in [0.001, 0.5, 1, 5, 10, 100, 500, 1000]:
            print('C='+str(C))
            model = LogisticRegression(C=C)
            model.fit(X_train, y_train)
            english_prediction = model.predict(X_test)
            all_valid_prediction = model.predict(X_valid_all)
            translated_valid_prediction = model.predict(X_valid_translated)
            english_roc = sklearn.metrics.roc_auc_score(y_test, english_prediction)
            all_valid_roc = sklearn.metrics.roc_auc_score(y_valid_all, all_valid_prediction)
            translated_valid_roc = sklearn.metrics.roc_auc_score(y_valid_translated, translated_valid_prediction)
            print('english: ', english_roc, 'all_v: ', all_valid_roc, 'translated_v: ', translated_valid_roc)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=750,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001
english:  0.5 all_v:  0.5 translated_v:  0.5
C=0.5
english:  0.6326854349166723 all_v:  0.5010718017076774 translated_v:  0.5004916905128769
C=1


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.6789039223116303 all_v:  0.5039173301629619 translated_v:  0.5047289786484702
C=5
english:  0.737950352554815 all_v:  0.5087593519952924 translated_v:  0.5092678296279992
C=10
english:  0.7386153400352183 all_v:  

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5088344081372868 translated_v:  0.5105527734976888
C=100
english:  0.7399490300098819 all_v:  0.5118018277671699 translated_v:  0.5074691833590139
C=500


...

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

 0.735739919309899 all_v:  0.5147536357195182 translated_v:  0.5147025643847678
C=100
english:  0.7350823618572097 all_v:  0.5234217194461457 translated_v:  0.5246563394232886
C=500


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


english:  0.7364123368180163 all_v:  0.537228446878265 translated_v:  0.5412293638564825
C=1000
english:  0.7375194109474027 all_v:  0.5462958292803016 translated_v:  0.5456325665859564


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Результаты того же порядка, соотношение такое же: сильно лучше на оригинале.
Будем обучатьсяна двойных переводах

In [56]:
for ngram_range in [(1, 1), (1,3)]:
    for max_features in [750, 1500]:
        tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
        print(tfidf)
        X_train = tfidf.fit_transform(bad_english_train)
        X_test = tfidf.transform(bad_english_test)
        X_original = tfidf.transform(original_test)
        X_valid_all= tfidf.transform(texts_valid_all)
        X_valid_translated = tfidf.transform(texts_valid_translated)
        for C in [0.001, 0.5, 1, 5, 10, 100, 500, 1000]:
            print('C='+str(C))
            model = LogisticRegression(C=C)
            model.fit(X_train, y_train)
            bad_english_prediction = model.predict(X_test)
            original_english_prediction = model.predict(X_original)
            all_valid_prediction = model.predict(X_valid_all)
            translated_valid_prediction = model.predict(X_valid_translated)
            bad_english_roc = sklearn.metrics.roc_auc_score(y_test, bad_english_prediction)
            original_english_roc = sklearn.metrics.roc_auc_score(y_test, original_english_prediction)
            all_valid_roc = sklearn.metrics.roc_auc_score(y_valid_all, all_valid_prediction)
            translated_valid_roc = sklearn.metrics.roc_auc_score(y_valid_translated, translated_valid_prediction)
            print('original_english:', original_english_roc,'bad_english: ', bad_english_roc, 'all_v: ', all_valid_roc, 'translated_v: ', translated_valid_roc)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=750,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001
original_english: 0.5 bad_english:  0.5 all_v:  0.5 translated_v:  0.5
C=0.5
original_english: 0.6172049721745463 bad_english:  0.5915528014919496 all_v:  0.5006652976426368 translated_v:  0.5004916905128769
C=1
original_english: 0.6691743010201429 bad_english:  0.6196383062508823 all_v:  0.5052857537438004 translated_v:  0.5053321043363416
C=5
original_english: 0.6965985333125292 bad_english:  0.665195521179294 all_v:  0.5225894969437138 translated_v:  0.5205263592339864
C=10
original_english

C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

original_english: 0.7138547726783021 bad_english:  0.6952797033932937 all_v:  0.5432521526101524 translated_v:  0.5391861104996698
C=500
original_english: 0.7114251536158231 bad_english:  0.6809100297944111 all_v:  0.5484400331447923 translated_v:  0.5432855491965661
C=1000


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


original_english: 0.7120901410962264 bad_english:  0.6811329306258312 all_v:  0.5411337680585078 translated_v:  0.5319175104556461
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1500,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
C=0.001
original_english: 0.5 bad_english:  0.5 all_v:  0.5 translated_v:  0.5
C=0.5
original_english: 0.616762885525563 bad_english:  0.5957544821642184 all_v:  0.5007391528863591 translated_v:  0.5006053268765133
C=1
original_english: 0.6654147069968571 bad_english:  0.6368871155889411 all_v:  0.5042505794334162 translated_v:  0.5041979418886199
C=5
original_english: 0.

...

original_english: 0.7145123301309914 bad_english:  0.6813409714018233 all_v:  0.527956911770004 translated_v:  0.5253929121725732
C=100
original_english: 0.7105372653039996 bad_english:  0.6749362875123525 all_v:  0.5507823852241477 translated_v:  0.5443605546995378
C=500


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


original_english: 0.708991819539487 bad_english:  0.6716224951519071 all_v:  0.5544096984544439 translated_v:  0.5472053158705701
C=1000


C:\Users\lizan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


original_english: 0.7143005743411422 bad_english:  0.6826820914042008 all_v:  0.5562716912250364 translated_v:  0.5516456636583755


На английском даже при обучении на дважды переведенных текстах лучше работает на оригинальных.На валидации от использования двойного перевода качество повысилось, но не сильно. 
На дважды переведенных английских работает сильно лучше, чем на переведенных с других языков
